In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0=all logs, 1=no INFO, 2=no WARN, 3=no ERROR
import tensorflow as tf

In [2]:
# example of loading the mnist dataset
from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical  # For one-hot encoding
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std
import numpy as np, random
from sklearn.model_selection import train_test_split

In [3]:
import matplotlib.pyplot as plt

def plot_data(loss_data, accuracy_data, path):
    plt.figure(figsize=(8, 5))
    plt.plot(loss_data, label='loss', color='blue')
    plt.xlabel('Iteration')
    plt.ylabel('Loss', color='blue')
    plt.tick_params(axis='y', labelcolor='blue')

    plt.twinx()
    plt.plot(accuracy_data, label='accuracy', color='green')
    plt.ylabel('Accuracy', color='green')
    plt.tick_params(axis='y', labelcolor='green')

    plt.title('Loss and accuracy over the time')
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path)
    plt.legend(loc='upper left')
    plt.show()

In [4]:
# load train and test dataset
def load_dataset():
    # 1. Load MNIST data
    (X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

    # 2. Normalize and reshape for CNN (add channel dimension)
    X_train_full = X_train_full.reshape((X_train_full.shape[0], 28, 28, 1)).astype('float32') / 255.0
    X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

    # 3. Split into train (60%) + validation (20%)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full,
        test_size=0.25,  # 25% of 60k = 15k samples
        random_state=42
    )

    # 4. One-hot encode labels
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    y_test = to_categorical(y_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [5]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [6]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [7]:
def evaluate_model(model, X_test, y_test):
    # evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return loss, accuracy

In [8]:
# evaluate a model using k-fold cross-validation
def evaluate_model_initial(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=100, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        loss, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        print(f"Loss/Fitness = {loss}, Accuracy = {acc}")  # Fixed: Consistent indentation (spaces)
        # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories, model

In [9]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(2, 1, 1)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(2, 1, 2)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()

In [10]:
# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY, valX, valY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # evaluate model
    scores, histories, model = evaluate_model_initial(trainX, trainY)
    # print the weights
    get_weights_model(model)
    # learning curves
    summarize_diagnostics(histories)
    return model, trainX, trainY, testX, testY, valX, valY

In [11]:
def setup_run():
    model, trainX, trainY, testX, testY, valX, valY = run_test_harness()

In [12]:
# entry point, run the test harness
setup_run()

C:\Users\pp1549\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


> 97.222
Loss/Fitness = 0.09585288166999817, Accuracy = 0.9722222089767456
> 96.889
Loss/Fitness = 0.10617673397064209, Accuracy = 0.9688888788223267
> 97.600
Loss/Fitness = 0.08597627282142639, Accuracy = 0.9760000109672546


KeyboardInterrupt: 